In [ ]:
# Import required libraries
from huggingface_hub import HfApi, CommitOperationDelete
import os

# Initialize API
api = HfApi()

# Set repository details
repo_id = "sirbastiano94/MAYA4"  # Replace with your repository ID

print("✅ Libraries imported successfully")
print(f"🎯 Target repository: {repo_id}")

In [ ]:
try:
    # List all files in the repository to see the structure
    files = api.list_repo_files(repo_id=repo_id, repo_type="dataset")
    print(f"📁 {len(files)} Files and folders in the repository")
        
except Exception as e:
    print(f"❌ Failed to list repository contents: {str(e)}")


In [ ]:
# Delete all files from the repository

try:
    print(f"🗑️ Starting deletion of {len(files)} files from repository {repo_id}...")
    
    # Create delete operations for all files (excluding .gitattributes)
    delete_operations = []
    files_to_delete = [f for f in files if f != '.gitattributes']
    
    for file_path in files_to_delete:
        delete_operations.append(CommitOperationDelete(path_in_repo=file_path))
    
    print(f"📝 Prepared {len(delete_operations)} delete operations")
    
    # Execute the deletion in batches to avoid API limits
    batch_size = 50
    total_batches = (len(delete_operations) + batch_size - 1) // batch_size
    
    for i in range(0, len(delete_operations), batch_size):
        batch = delete_operations[i:i + batch_size]
        batch_num = i // batch_size + 1
        
        print(f"🔄 Processing batch {batch_num}/{total_batches} ({len(batch)} files)...")
        
        api.create_commit(
            repo_id=repo_id,
            repo_type="dataset",
            operations=batch,
            commit_message=f"Delete batch {batch_num}/{total_batches} of files"
        )
        
        print(f"✅ Batch {batch_num} completed successfully")
    
    print(f"\n🎉 All files deleted successfully from {repo_id}!")
    
except Exception as e:
    print(f"❌ Failed to delete files: {str(e)}")
    print(f"💡 Error details: {type(e).__name__}")

In [ ]:
# Verify deletion by listing remaining files
try:
    remaining_files = api.list_repo_files(repo_id=repo_id, repo_type="dataset")
    print(f"📊 Repository status after deletion:")
    print(f"   Total remaining files: {len(remaining_files)}")
    
    if remaining_files:
        print(f"\n📁 Remaining files:")
        for file in remaining_files:
            print(f"   {file}")
    else:
        print(f"\n✨ Repository is now empty (except for system files)!")
        
except Exception as e:
    print(f"❌ Failed to verify deletion: {str(e)}")